In [26]:
import requests

# taiwan_cities_list = ["基隆市", "新北市", "宜蘭縣", "桃園市", "新竹縣", "新竹市", "苗栗縣", "臺中市", "南投縣", "彰化縣", "雲林縣", "嘉義縣", "嘉義市", "臺南市", "高雄市", "屏東縣", "臺東縣", "花蓮縣", "澎湖縣"]

headers = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-language': 'zh-TW,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'content-type': 'application/json',
    'origin': 'https://web.water.gov.tw',
    'priority': 'u=1, i',
    'referer': 'https://web.water.gov.tw/wateroff/city/%E8%87%BA%E4%B8%AD%E5%B8%82/index.html',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36 Edg/136.0.0.0',
    'x-requested-with': 'XMLHttpRequest',
}

json_data = {
    'mode': 1,
    'startDate': '2025-05-15',
    'endDate': '2025-06-14',
}

response = requests.post('https://web.water.gov.tw/wateroffapi/f/case/search', headers=headers, json=json_data)

In [ ]:
import json

def find_matching_outages(data_list, affectedCounties, affectedTowns=None):
    """
    根據受影響的縣市和可選的鄉鎮市區篩選停水/降壓資料。

    Args:
        data_list (list): 包含停水/降壓事件的字典列表。
        affectedCounties (str or list/tuple/set of str): 
            要篩選的縣市代碼。必須是非空字串或非空字串的非空集合。
        affectedTowns (str or list/tuple/set of str, optional): 
            要篩選的鄉鎮市區代碼。預設為 None (不按鄉鎮市區篩選)。
            如果提供空字串 "" 或空集合 []/{}/()，則會匹配 'affectedTowns' 欄位也為空的項目。
            如果提供非空字串或非空集合，則所有元素必須是非空字串。

    Returns:
        list: 一個新的列表，其中只包含符合條件的字典。
    
    Raises:
        TypeError: 如果篩選條件的輸入類型不正確。
        ValueError: 如果 `affectedCounties` 為空或包含無效元素，
                    或者 `affectedTowns` (如果提供) 包含無效元素。
    """
    filtered_results = []

    # --- 驗證和標準化 affectedCounties 篩選條件 ---
    target_counties_set = set()
    if isinstance(affectedCounties, str):
        if not affectedCounties:  # 空字串
            raise ValueError("`affectedCounties` 字串篩選條件不可為空。")
        target_counties_set = {affectedCounties}
    elif isinstance(affectedCounties, (list, tuple, set)):
        if not affectedCounties:  # 空集合
            raise ValueError("`affectedCounties` 集合篩選條件不可為空。")
        # 檢查集合內是否有非字串或空字串元素
        for c in affectedCounties:
            if not isinstance(c, str) or not c:
                raise ValueError("`affectedCounties` 集合中的所有元素都必須是非空字串。")
        target_counties_set = set(affectedCounties)
    else:
        raise TypeError(
            "`affectedCounties` 必須是非空字串或非空字串的非空集合 (list, tuple, set)。"
        )

    # --- 驗證和標準化 affectedTowns 篩選條件 (如果提供) ---
    target_towns_set = None  # 預設：不按鄉鎮市區篩選
    if affectedTowns is not None:
        if isinstance(affectedTowns, str):
            # 如果鄉鎮市區為空字串，表示 "匹配沒有鄉鎮市區的項目"
            # 因此，target_towns_set 變成一個空集合
            target_towns_set = {affectedTowns} if affectedTowns else set()
        elif isinstance(affectedTowns, (list, tuple, set)):
            # 如果 affectedTowns 是空列表/元組/集合，target_towns_set 變成空集合。
            # 這表示 "僅匹配 'affectedTowns' 欄位也為空的項目"。
            # 僅當集合非空時才驗證其元素
            if affectedTowns: 
                for t in affectedTowns: 
                    if not isinstance(t, str) or not t:
                        raise ValueError("非空的 `affectedTowns` 集合中的所有元素都必須是非空字串。")
            target_towns_set = set(affectedTowns)
        else:
            raise TypeError(
                "`affectedTowns` 必須是字串、字串集合 (list, tuple, set) 或 None。"
            )

    # --- 迭代並篩選資料 ---
    for item in data_list:
        item_actual_counties = set(item.get('affectedCounties', []))
        
        # 1. 縣市匹配:
        # target_counties_set 和 item_actual_counties 之間必須有交集。
        if not (target_counties_set & item_actual_counties):
            continue  # 沒有匹配的縣市

        # 2. 鄉鎮市區匹配 (如果鄉鎮市區篩選條件啟用):
        if target_towns_set is not None:
            item_actual_towns = set(item.get('affectedTowns', []))
            
            # 不匹配的情況發生於:
            # A) target_towns_set 為空 (篩選條件是 "沒有鄉鎮市區") 且 item_actual_towns 非空。
            # 或
            # B) target_towns_set 非空 (篩選條件是特定的鄉鎮市區) 且兩者之間沒有交集。
            if (not target_towns_set and item_actual_towns) or \
               (target_towns_set and not (target_towns_set & item_actual_towns)):
                continue # 沒有匹配的鄉鎮市區

        # 如果所有條件都通過
        filtered_results.append(item)
            
    return filtered_results

# --- 使用範例 ---
with open("water_outage_notices.json", "r") as f:
    data = json.load(f)

results1 = find_matching_outages(data,"64000")
print(f"\n測試 1: 縣市 '66000'。找到: {len(results1)} 項")
for r in results1: print(f"  ID: {r['id']}, 縣市: {r['affectedCounties']}, 鄉鎮: {r.get('affectedTowns')}")


results2 = find_matching_outages(data, affectedCounties="66000", affectedTowns="66000170")
print(f"\n測試 2: 縣市 '66000', 鄉鎮 '66000170'。找到: {len(results2)} 項")
for r in results2: print(f"  ID: {r['id']}, 縣市: {r['affectedCounties']}, 鄉鎮: {r.get('affectedTowns')}")


# # 測試 3: 按縣市和鄉鎮列表篩選 (列表中各有一個鄉鎮匹配每個項目)
results3 = find_matching_outages(data, affectedCounties="66000", affectedTowns=["66000170", "66000270"])
print(f"\n測試 3: 縣市 '66000', 鄉鎮 ['66000170', '66000270']。找到: {len(results3)} 項")
for r in results3: print(f"  ID: {r['id']}, 縣市: {r['affectedCounties']}, 鄉鎮: {r.get('affectedTowns')}")

# 測試 4: 按縣市篩選，但鄉鎮不在資料中
results4 = find_matching_outages(data, affectedCounties="66000", affectedTowns="99999")
print(f"\n測試 4: 縣市 '66000', 鄉鎮 '99999'。找到: {len(results4)} 項")
# 預期: 0 項

# 測試 5: 按不在資料中的縣市篩選
results5 = find_matching_outages(data, affectedCounties="00000")
print(f"\n測試 5: 縣市 '00000'。找到: {len(results5)} 項")
# 預期: 0 項

# 測試 6: 按縣市篩選，鄉鎮為空列表 (應匹配沒有鄉鎮的項目)
# 使用擴展的資料列表進行此測試
results6 = find_matching_outages(data, affectedCounties="XYZ", affectedTowns=[])
print(f"\n測試 6: 縣市 'XYZ', 鄉鎮 []。找到: {len(results6)} 項")
for r in results6: print(f"  ID: {r['id']}, 縣市: {r['affectedCounties']}, 鄉鎮: {r.get('affectedTowns', '無鄉鎮欄位')}")
# 預期: 2 項 (ID 999999 affectedTowns為[], ID 999998 無affectedTowns欄位)

# 測試 7: 按縣市篩選，鄉鎮為空字串 (也應匹配沒有鄉鎮的項目)
results7 = find_matching_outages(data, affectedCounties="XYZ", affectedTowns="")
print(f"\n測試 7: 縣市 'XYZ', 鄉鎮 \"\"。找到: {len(results7)} 項")
for r in results7: print(f"  ID: {r['id']}, 縣市: {r['affectedCounties']}, 鄉鎮: {r.get('affectedTowns', '無鄉鎮欄位')}")
# 預期: 2 項

# 測試 8: 篩選條件中有多個縣市
multi_county_data = data + [{"id":111, "affectedCounties":["77000"], "affectedTowns":["77000010"], "note":"..."}]
results8 = find_matching_outages(multi_county_data, affectedCounties=["66000", "77000"])
print(f"\n測試 8: 縣市 ['66000', '77000']。找到: {len(results8)} 項")
for r in results8: print(f"  ID: {r['id']}, 縣市: {r['affectedCounties']}, 鄉鎮: {r.get('affectedTowns')}")
# 預期: 3 項 (2 個原始 + 1 個新的)

# --- 錯誤處理測試 ---
print("\n--- 錯誤處理測試 ---")
try:
    find_matching_outages(data, affectedCounties="")
except ValueError as e:
    print(f"測試 9.1 (空縣市字串): 通過 - {e}")
try:
    find_matching_outages(data, affectedCounties=[])
except ValueError as e:
    print(f"測試 9.2 (空縣市列表): 通過 - {e}")
try:
    find_matching_outages(data, affectedCounties=["66000", ""]) # 縣市列表中的空字串
except ValueError as e:
    print(f"測試 9.3 (縣市列表含空字串): 通過 - {e}")
try:
    find_matching_outages(data, affectedCounties=123) # 縣市類型錯誤
except TypeError as e:
    print(f"測試 9.4 (縣市類型錯誤): 通過 - {e}")
try:
    find_matching_outages(data, affectedCounties="66000", affectedTowns=123) # 鄉鎮類型錯誤
except TypeError as e:
    print(f"測試 9.5 (鄉鎮類型錯誤): 通過 - {e}")
try:
    # 鄉鎮列表中包含非字串元素 (且鄉鎮列表非空)
    find_matching_outages(data, affectedCounties="66000", affectedTowns=["66000170", 123]) 
except ValueError as e: 
    print(f"測試 9.6 (鄉鎮列表含非字串): 通過 - {e}")
try:
    # 鄉鎮列表中包含空字串元素 (且鄉鎮列表非空)
    find_matching_outages(data, affectedCounties="66000", affectedTowns=["66000170", ""]) 
except ValueError as e: 
    print(f"測試 9.7 (鄉鎮列表含空字串): 通過 - {e}")


測試 1: 縣市 '66000'。找到: 13 項
  ID: 141409, 縣市: ['64000'], 鄉鎮: ['64000210']
  ID: 141432, 縣市: ['64000'], 鄉鎮: ['64000220']
  ID: 141462, 縣市: ['64000'], 鄉鎮: ['64000090']
  ID: 141439, 縣市: ['64000'], 鄉鎮: ['64000300']
  ID: 141442, 縣市: ['64000'], 鄉鎮: ['64000270']
  ID: 141444, 縣市: ['64000'], 鄉鎮: ['64000180']
  ID: 141408, 縣市: ['64000'], 鄉鎮: ['64000210']
  ID: 141031, 縣市: ['64000'], 鄉鎮: ['64000210']
  ID: 141206, 縣市: ['64000'], 鄉鎮: ['64000040', '64000160']
  ID: 141405, 縣市: ['64000'], 鄉鎮: ['64000030']
  ID: 141404, 縣市: ['64000'], 鄉鎮: ['64000240']
  ID: 141407, 縣市: ['64000'], 鄉鎮: ['64000040']
  ID: 141420, 縣市: ['64000'], 鄉鎮: ['64000190']

測試 2: 縣市 '66000', 鄉鎮 '66000170'。找到: 5 項
  ID: 140236, 縣市: ['66000'], 鄉鎮: ['66000170', '66000080']
  ID: 138321, 縣市: ['66000'], 鄉鎮: ['66000170', '66000080']
  ID: 141383, 縣市: ['66000'], 鄉鎮: ['66000170']
  ID: 141438, 縣市: ['66000'], 鄉鎮: ['66000170', '66000090']
  ID: 141424, 縣市: ['66000'], 鄉鎮: ['66000170', '66000180']

測試 3: 縣市 '66000', 鄉鎮 ['66000170', '66000270